In [1]:
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

''' 
This program merges all of the separate programs (protagonists.ipynb and synopsis.ipynb) and adds the text search function. It will
return all of the movies that contain family keywords inside a data frame. This program runs in approximately 7-8 mins."
'''

def movie_table(url):
    webpage = url
    response = requests.get(webpage)

    page = response.text

    soup = BeautifulSoup(page, 'html.parser')
    beautiful_page = soup.get_text()

    ratings = []
    years = []
    titles = []
    movie_nums = []

    lines = beautiful_page.splitlines()

    for line in lines:
        if "(20" in line and ")" in line or '(19' in line and ')' in line:
            years.append(line)

    rep_years = []
    nrep_years = []
    year_indices = []

    for year in years:
        
        if lines.count(year) > 1:
            if year not in rep_years:
                rep_years.append(year)
        if lines.count(year) == 1:
            nrep_years.append(year)

    for i in range(len(rep_years)):
        first_instance = lines.index(rep_years[i])
        year_indices.append(first_instance)

        for j, item in enumerate(lines[first_instance+1:], start=first_instance+1):
            if item == rep_years[i]:
                year_indices.append(j)

    for year in nrep_years:
        if year in lines:
            index = lines.index(year)
            year_indices.append(index)


    for index in sorted(year_indices):
        title = lines[index-1]
        titles.append(title)

        rating = lines[index+6]
        ratings.append(rating)

        movie_num = lines[index-3]
        movie_nums.append(movie_num) 
        

    ratings = [rating.lstrip() for rating in ratings]
    for i in range(len(ratings)):
        if ratings[i] == '':
            ratings[i] = "None"


    for i in range(len(years)):
        for j in range(len(years[i])):
            if years[i][j].isnumeric():
                number_index = years[i].find(years[i][j])
                years[i] = years[i][number_index:number_index+4]
                break


    table_data = list(zip(movie_nums, titles, years, ratings))
    # headers = ["Movie Number", "Title", "Year", "Rating"]

    # print(tabulate(table_data, headers=headers))
    
    return table_data

synopsis_list = []

# Function to get the synopsis from every movie in the webpage
def synopses(link):
    webpage = link
    response = requests.get(webpage)

    page = response.text

    soup = BeautifulSoup(page, 'html.parser')

    url_list = []
    movie_url_list = []
    movie_ref_nums = []

    # Get all the hyperlinks
    for link in soup.find_all('a'):
        url_list.append(link.get('href'))

    # Filter to get movie title hyperlinks
    for url in url_list:
        if '/title/tt' in url:
            if url not in movie_url_list:
                movie_url_list.append(url) 

    # Take the reference number from the movie url
    for movie_url in movie_url_list:
        movie_ref_nums.append(movie_url[7:16])


    # Extract the synopsis from every movie then append to a list
    driver = webdriver.Chrome('/Users/nixoncarino/Desktop/chromedriver')

    for i in movie_ref_nums:
        url = "https://www.imdb.com/title/" + i + "/plotsummary/?ref_=tt_stry_pl#synopsis"
        try:
            driver.get(url)
            synopsis_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/div[2]/ul/li/div/div/div/div/div')
        except:
            synopsis_list.append("No Synopsis")
        else:
            synopsis = synopsis_element.text
            synopsis_list.append(synopsis)

# Function that find family related words on the synopsis
def text_search(synopsis_list):
    # The indices of the synopses that contain any of the keywords
    match_index_list = []

    keywords = ["family", "parenting", "father", "mother", "son", "daughter", "sibling", "brother", "sister", "marriage", "divorce", "blended family", "adoption",
                "foster care", "custody", "inheritance", "legacy", "generations", "multigenerational", "family reunion", "family secrets", "family drama", "dysfunctional", 
                "dysfunctional family", "family conflict", "family relationships", "family values", "family ties", "parent-child relationship", "relationship",
                "coming-of-age", "parenthood", "growing up", "childhood", "teenage years", "aging parents", "grandparents"]

    for i in range(len(synopsis_list)):
        for keyword in keywords:
            if keyword in synopsis_list[i]:
                if i not in match_index_list:
                    match_index_list.append(i)

    return match_index_list

def non_family_movies(synopsis_list):
    all_indices = list(range(0,len(synopsis_list)))

    # The indices of the synopses that contain any of the keywords
    match_index_list = []

    keywords = ["family", "parenting", "father", "mother", "son", "daughter", "sibling", "brother", "sister", "marriage", "divorce", "blended family", "adoption",
                "foster care", "custody", "inheritance", "legacy", "generations", "multigenerational", "family reunion", "family secrets", "family drama", "dysfunctional", 
                "dysfunctional family", "family conflict", "family relationships", "family values", "family ties", "parent-child relationship", "relationship",
                "coming-of-age", "parenthood", "growing up", "childhood", "teenage years", "aging parents", "grandparents"]

    for i in range(len(synopsis_list)):
        for keyword in keywords:
            if keyword in synopsis_list[i]:
                if i not in match_index_list:
                    match_index_list.append(i)
    
    non_match_index = [index for index in all_indices if index not in match_index_list]

    return non_match_index

# Call synopses function to put all synopses into list
synopses("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=superhero&view=simple")
synopses("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&view=simple&start=51&ref_=adv_nxt")

# Create Data Frame for Movie information
df_superhero_1 = pd.DataFrame(movie_table("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=superhero&view=simple"), 
                            columns=['Index', 'Title', 'Year', 'Rating'])
df_superhero_2 = pd.DataFrame(movie_table("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&view=simple&start=51&ref_=adv_nxt"), 
                            columns=['Index', 'Title', 'Year', 'Rating'])
df_superhero = pd.concat([df_superhero_1, df_superhero_2], ignore_index=True).assign(Genre='Superhero', Synopsis=synopsis_list)

# Output information of movies that contain family keywords
pd.set_option('display.max_rows', None)
df_superhero.iloc[text_search(synopsis_list)]


/var/folders/dr/q8lk9535757d9krftyrx8kgc0000gn/T/ipykernel_22573/485644917.py:122: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/nixoncarino/Desktop/chromedriver')


,Index,Title,Year,Rating,Genre,Synopsis
0,1.,Guardians of the Galaxy Vol. 3,2023,8.3,Superhero,"As a baby raccoon, Rocket is a test subject fo..."
2,3.,The Super Mario Bros. Movie,2023,7.2,Superhero,The movie starts with Bowser's ship landing in...
3,4.,Dungeons & Dragons: Honor Among Thieves,2023,7.4,Superhero,"Prior to imprisonment, bard Edgin Darvis serve..."
7,8.,A Man Called Otto,2022,7.5,Superhero,"Otto Anderson, a 63-year-old widower, lives in..."
11,12.,Guardians of the Galaxy,2014,8,Superhero,"Earth, 1988:\n\nYoung Peter Quill (Wyatt Oleff..."
15,16.,F9: The Fast Saga,2021,5.2,Superhero,"In 1989, Jack Toretto (JD Pardo) participates ..."
16,17.,Guardians of the Galaxy Vol. 2,2017,7.6,Superhero,"Missouri, Planet Earth, 1980\n\nMeredith Quill..."
17,18.,Avatar: The Way of Water,2022,7.7,Superhero,Over a decade after the events of the first fi...
19,20.,Heat,1995,8.3,Superhero,An inbound Los Angeles Blue Line train pulls i...
21,22.,Everything Everywhere All at Once,2022,7.8,Superhero,Part 1: Everything\n\nEvelyn Wang (Yeoh) is a ...


In [2]:
non_family = df_superhero.iloc[non_family_movies(synopsis_list)]; non_family

,Index,Title,Year,Rating,Genre,Synopsis
1,2.,Air,2023,7.5,Superhero,No Synopsis
4,5.,Evil Dead Rise,2023,6.8,Superhero,No Synopsis
5,6.,Guy Ritchie's the Covenant,2023,7.6,Superhero,No Synopsis
6,7.,Ant-Man and the Wasp: Quantumania,2023,6.2,Superhero,No Synopsis
8,9.,John Wick: Chapter 4,2023,8.2,Superhero,No Synopsis
9,10.,The Pope's Exorcist,2023,6.1,Superhero,No Synopsis
10,11.,Scream VI,2023,6.7,Superhero,No Synopsis
12,13.,Renfield,2023,6.4,Superhero,No Synopsis
13,14.,Ghosted,2023,5.8,Superhero,No Synopsis
14,15.,The Kerala Story,2023,7.5,Superhero,No Synopsis
